This notebook uses the network created by ```Topology_clique_activity_experiment.ipynb```, and ablates all connections to run a reference unconnected run.

In [1]:
import os
os.environ["SNUDDA_DATA"] = "../../../../BasalGangliaData/data/"
assert os.path.isdir(os.getenv("SNUDDA_DATA")), f"You need to have BasalGangliaData installed for this example."

network_path = "topology100"
network_file = os.path.join(network_path, "network-synapses.hdf5")
simplex_file_name = os.path.join(network_path, "topology100_D6_simplices0.simplices")
ablated_network_file = os.path.join(network_path, "network-fully-ablated.hdf5")

remove_sink_to_source_connections = True

In [2]:
from snudda.analyse.analyse_topology import SnuddaAnalyseTopology
sat = SnuddaAnalyseTopology(network_file=network_file)
sat.load_simplex_file(simplex_file_name=simplex_file_name)
mult = sat.get_fixed_multiplicity()

Loading 99 neurons with 6999 synapses and 0 gap junctions
Loading 99 neurons with 6999 synapses and 0 gap junctions
Loaded simplex data of dimension 6 from topology100/topology100_D6_simplices0.simplices


Next in this example we list the cliques with multiplicity 3. To see all of them simply inspect mult[6]. Here the 6 refers to the clique dimension we loaded. You could also load multiple clique dimension files at the same time and compare them.

In [ ]:
[(x[0], x[1]) for x in mult[6].items() if x[1] == 3]

In [ ]:
selected_clique_neurons = list(mult[6])[0]

In [ ]:
selected_clique_neurons

### Ablating network

In [ ]:
from snudda.utils.ablate_network import SnuddaAblateNetwork
san = SnuddaAblateNetwork(network_file=network_file)
san.only_keep_neuron_id(selected_clique_neurons)

The cell below removes sink to source connections if the ```remove_sink_to_source``` flag is set.

In [ ]:
san.ablate_all_synapses()

Note that the neurons get remapped, when we ablate the network and remove a subset of the neurons.

In [ ]:
san.write_network(out_file_name=ablated_network_file, print_remapping=True)

### Simulating network

Next step is running the network with current injections (we should also run it with synaptic input).

#### NEED TO CHANGE TO BasalGangliaData

In [ ]:
experiment_config_file = "topology_clique_simulation_experiment-6a.json"
from snudda.simulate.pair_recording import PairRecording 
pr = PairRecording(network_path=network_path, network_file=ablated_network_file,
                   experiment_config_file=experiment_config_file)

In [ ]:
pr.run()

In [ ]:
%matplotlib notebook
pr.plot_trace_overview()

In [ ]:
pr.plot_traces()